## File to implement nn with wandb
(note that we can't run wandb in terminal)

In [1]:
# file to load the nn

import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import wandb
from wandb.keras import WandbCallback

# read in our data
DATA_DIR = '/home/oscar47/Desktop/astro101/data/g_band/var_output/'

# check if keras recognizes gpu
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

train_x_ds = np.load(os.path.join(DATA_DIR, 'train_x_ds.npy'))
val_x_ds = np.load(os.path.join(DATA_DIR, 'val_x_ds.npy'))
train_y_ds = np.load(os.path.join(DATA_DIR, 'train_y_ds.npy'))
val_y_ds = np.load(os.path.join(DATA_DIR, 'val_y_ds.npy'))

input_shape = train_x_ds[0].shape
output_len = len(train_y_ds[0])

# build model functions--------------------------------
def build_model(size1, size2, size3, size4, size5, dropout, learning_rate):
    model = Sequential()

    model.add(layers.Dense(size1))
    model.add(layers.Dense(size2))
    model.add(layers.Dense(size3))
    model.add(layers.Dense(size4))
    model.add(layers.Dense(size5))

    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(output_len))

    # return len of class size
    model.add(layers.Dense(output_len))
    model.add(layers.Activation('softmax'))

    optimizer = Adam(learning_rate = learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')

    return model


def train(config=None):
    with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
      config = wandb.config

      #pprint.pprint(config)

      #initialize the neural net; 
      global model
      model = build_model(config.size_1,  config.size_2, config.size_3, 
              config.size_4, config.size_5, 
              config.dropout, config.learning_rate)
      
      #now run training
      history = model.fit(
        train_x_ds, train_y_ds,
        batch_size = config.batch_size,
        validation_data=(val_x_ds, val_y_ds),
        epochs=config.epochs,
        callbacks=[WandbCallback()] #use callbacks to have w&b log stats; will automatically save best model                     
      )

def train_manual():
    global model
    model = build_model(128, 128, 128, 
            128, 128, 
            .1, .001)
    
    #now run training
    history = model.fit(
    train_x_ds, train_y_ds,
    batch_size = 64,
    validation_data=(val_x_ds, val_y_ds),
    epochs=10
    )

# set dictionary with random search; optimizing val_loss--------------------------
sweep_config= {
    'method': 'random',
    'name': 'val_accuracy',
    'goal': 'maximize'
}

sweep_config['metric']= 'val_accuracy'

# now name hyperparameters with nested dictionary
# parameters_dict = {
#     'epochs': {
#        'distribution': 'int_uniform',
#        'min': 10,
#        'max': 20
#     },
#     # for build_dataset
#      'batch_size': {
#        'distribution': 'q_log_uniform',  #we want to specify a distribution type to more efficiently iterate through these hyperparams
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#     'size_1': {
#        'distribution': 'q_log_uniform',
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#     'size_2': {
#        'distribution': 'q_log_uniform',
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#      'size_3': {
#        'distribution': 'q_log_uniform',
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#      'size_4': {
#        'distribution': 'q_log_uniform',
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#      'size_5': {
#        'distribution': 'q_log_uniform',
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#     'dropout': {
#       'distribution': 'uniform',
#        'min': 0,
#        'max': 0.6
#     },
#     'learning_rate':{
#          #uniform distribution between 0 and 1
#          'distribution': 'uniform', 
#          'min': 0,
#          'max': 0.1
#      }
# }

parameters_dict = {
    'epochs': {
       'distribution': 'int_uniform',
       'min': 20,
       'max': 100
    },
    # for build_dataset
     'batch_size': {
       'values': [32, 64, 96, 128]
    },
    'size_1': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },
    'size_2': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },'size_3': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },'size_4': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },'size_5': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },
    'dropout': {
      'distribution': 'uniform',
       'min': 0,
       'max': 0.6
    },
    'learning_rate':{
         #uniform distribution between 0 and 1
         'distribution': 'uniform', 
         'min': 0,
         'max': 0.1
     }
}

# append parameters to sweep config
sweep_config['parameters'] = parameters_dict 

# login to wandb----------------
wandb.init(project="Astro101_Project_v2", entity="oscarscholin")

# initialize sweep agent
sweep_id = wandb.sweep(sweep_config, project='Astro101_Project_v2', entity="oscarscholin")
wandb.agent(sweep_id, train, count=100)

#train_manual()

2022-11-18 04:45:40.620608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 04:45:40.690566: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-18 04:45:40.692276: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-18 04:45:40.692282: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc

Num GPUs Available:  0


2022-11-18 04:45:41.565993: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 04:45:41.566456: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-18 04:45:41.566478: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-18 04:45:41.566493: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-18 04:45:41.566508: W tensorflow/c

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('goal' was unexpected)
wandb: WARNING   Violation 2. 'val_accuracy' is not of type 'object'
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: jp6ffgsr
Sweep URL: https://wandb.ai/oscarscholin/Astro101_Project_v2/sweeps/jp6ffgsr


...

wandb: Adding directory to artifact (/home/oscar47/Desktop/astro101/Astro101-Final-Project/nn_v0.0.1/wandb/run-20221118_105610-js7eruzy/files/model-best)... Done. 0.0s


1527/1527 [==============================] - 4s 3ms/step - loss: 1.5701 - val_loss: 0.8713
Epoch 4/80
1527/1527 [==============================] - 3s 2ms/step - loss: 3366100480.0000 - val_loss: 21947556.0000
Epoch 5/80
1527/1527 [==============================] - 3s 2ms/step - loss: 16187810.0000 - val_loss: 8920335.0000
Epoch 6/80
1527/1527 [==============================] - 3s 2ms/step - loss: 6743200.5000 - val_loss: 4030246.2500
Epoch 7/80
1527/1527 [==============================] - 3s 2ms/step - loss: 3400988.5000 - val_loss: 992067.6875
Epoch 8/80
1527/1527 [==============================] - 3s 2ms/step - loss: 1372907.2500 - val_loss: 402965.3438
Epoch 9/80
1527/1527 [==============================] - 3s 2ms/step - loss: 2294290944.0000 - val_loss: 43158900.0000
Epoch 10/80
1527/1527 [==============================] - 3s 2ms/step - loss: 15680252.0000 - val_loss: 4766388.0000
Epoch 11/80
1527/1527 [==============================] - 3s 2ms/step - loss: 5211546.5000 - val_loss: 

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▁▁▁▁▅▁▁▁▁▁▆▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▄▁█▁▁▄▁▁▄▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,2
best_val_loss,0.87131
epoch,79
loss,4003359.0
val_loss,1608270.875


Error in callback <function _WandbInit._pause_backend at 0x7f65a4f4a700> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe